In [1]:
import torch
import torch.nn.functional as F
import hashlib

In [2]:
class CortexTensorDemo:
    def __init__(self, n_basis=10, m_registers=8, b_bits=4, tau=0.6):
        self.n_basis = n_basis
        self.m = m_registers
        self.b = b_bits
        self.tau = tau
        
        # Initialize basis tensor: [n_basis, m, b] - binary HLLSets
        self.basis_tensor = torch.bernoulli(torch.ones(n_basis, m_registers, b_bits) * 0.3)
        
        # Initialize content store (simulated)
        self.content_store = {}
        
    def compute_id(self, tensor):
        """Compute content-addressable ID from tensor"""
        tensor_bytes = tensor.numpy().tobytes()
        return hashlib.sha256(tensor_bytes).hexdigest()[:16]
    
    def optimal_cover(self, hllset, top_k=3):
        """Find optimal cover of hllset by basis elements"""
        # Compute similarities with all basis elements
        similarities = []
        for i in range(self.n_basis):
            basis_vec = self.basis_tensor[i].flatten()
            input_vec = hllset.flatten()
            sim = torch.dot(input_vec, basis_vec) / (torch.norm(input_vec) * torch.norm(basis_vec) + 1e-8)
            similarities.append(sim)
        
        similarities = torch.tensor(similarities)
        
        # Select top_k most similar basis elements
        _, indices = torch.topk(similarities, top_k)
        
        # Create sparse i-context vector
        i_context = torch.zeros(self.n_basis)
        i_context[indices] = 1.0
        
        return i_context
    
    def create_edge_hllset(self, i_context_a, i_context_b):
        """Create Edge_HLLSet from intersection of i-contexts"""
        # Intersection of basis elements
        intersection_mask = (i_context_a * i_context_b) > 0
        
        if intersection_mask.sum() == 0:
            return None
            
        # Union of intersecting basis elements
        intersecting_basis = self.basis_tensor[intersection_mask]  # [k, m, b]
        edge_hllset = torch.clamp(intersecting_basis.sum(dim=0), 0, 1)  # Bitwise OR
        
        return edge_hllset
    
    def build_context_tensor(self, input_hllsets):
        """Build context tensor from input HLLSets"""
        context_vectors = []
        
        for hllset in input_hllsets:
            i_context = self.optimal_cover(hllset)
            context_vectors.append(i_context)
            
        return torch.stack(context_vectors)  # [V, n_basis]
    
    def compute_similarity_matrix(self, context_tensor):
        """Compute similarity matrix from context tensor"""
        # Normalize context vectors
        context_norm = F.normalize(context_tensor, p=2, dim=1)
        
        # Similarity matrix
        similarity = torch.mm(context_norm, context_norm.T)
        
        return similarity
    
    def build_edge_tensor(self, context_tensor, similarity_matrix):
        """Build edge tensor from context tensor and similarities"""
        V = context_tensor.shape[0]
        edge_tensor = torch.zeros(V, V, 4)  # [edge_hllset_id, tau, rho, phi_hash]
        
        for i in range(V):
            for j in range(i+1, V):
                if similarity_matrix[i, j] >= self.tau:
                    # Create edge
                    edge_hllset = self.create_edge_hllset(context_tensor[i], context_tensor[j])
                    
                    if edge_hllset is not None:
                        edge_id = self.compute_id(edge_hllset)
                        edge_tensor[i, j, 0] = float(int(edge_id, 16) % 10000)  # Simulated ID
                        edge_tensor[i, j, 1] = similarity_matrix[i, j]  # tau
                        edge_tensor[i, j, 2] = 1 - similarity_matrix[i, j]  # rho
                        edge_tensor[i, j, 3] = float(hash((i, j)) % 10000)  # phi hash
                        
                        # Store in content store
                        self.content_store[edge_id] = edge_hllset
        
        return edge_tensor
    
    def abstract_layer(self, context_tensor, edge_tensor):
        """Create next layer context tensor from edges"""
        V = context_tensor.shape[0]
        new_context_vectors = []
        
        for i in range(V):
            for j in range(i+1, V):
                if edge_tensor[i, j, 0] > 0:  # Valid edge
                    edge_id = str(int(edge_tensor[i, j, 0]))
                    edge_hllset = self.content_store.get(edge_id)
                    
                    if edge_hllset is not None:
                        # Compute i-context for the Edge_HLLSet
                        edge_i_context = self.optimal_cover(edge_hllset)
                        new_context_vectors.append(edge_i_context)
        
        if len(new_context_vectors) == 0:
            return None
            
        return torch.stack(new_context_vectors)
    
    def cortex_transformation(self, input_hllsets, num_layers=3):
        """Complete Cortex transformation pipeline"""
        print("=== Cortex Tensor Demo ===\n")
        
        # Store all layers
        layers = []
        
        # Layer 0: Process input HLLSets
        print(f"Layer 0: Processing {len(input_hllsets)} input HLLSets")
        context_tensor_0 = self.build_context_tensor(input_hllsets)
        similarity_0 = self.compute_similarity_matrix(context_tensor_0)
        edge_tensor_0 = self.build_edge_tensor(context_tensor_0, similarity_0)
        
        layers.append({
            'context': context_tensor_0,
            'similarity': similarity_0,
            'edges': edge_tensor_0
        })
        
        print(f"  Context tensor shape: {context_tensor_0.shape}")
        print(f"  Non-zero edges: {(edge_tensor_0[:, :, 0] > 0).sum().item()}")
        
        # Build higher layers
        current_context = context_tensor_0
        current_edges = edge_tensor_0
        
        for layer in range(1, num_layers):
            print(f"\nLayer {layer}: Abstracting from previous layer")
            
            next_context = self.abstract_layer(current_context, current_edges)
            
            if next_context is None or next_context.shape[0] < 2:
                print(f"  Termination: Not enough vertices for further abstraction")
                break
                
            next_similarity = self.compute_similarity_matrix(next_context)
            next_edges = self.build_edge_tensor(next_context, next_similarity)
            
            layers.append({
                'context': next_context,
                'similarity': next_similarity,
                'edges': next_edges
            })
            
            print(f"  Context tensor shape: {next_context.shape}")
            print(f"  Non-zero edges: {(next_edges[:, :, 0] > 0).sum().item()}")
            
            current_context = next_context
            current_edges = next_edges
        
        return layers

In [3]:
# Demo execution
def run_demo():
    # Initialize cortex
    cortex = CortexTensorDemo(n_basis=8, m_registers=6, b_bits=4, tau=0.5)
    
    # Create 2 random input HLLSets
    input_hllsets = [
        torch.bernoulli(torch.ones(6, 4) * 0.4),  # HLLSet A
        torch.bernoulli(torch.ones(6, 4) * 0.6)   # HLLSet B
    ]
    
    print("Input HLLSet A:")
    print(input_hllsets[0])
    print("\nInput HLLSet B:")
    print(input_hllsets[1])
    
    # Run cortex transformation
    layers = cortex.cortex_transformation(input_hllsets, num_layers=3)
    
    # Display results
    print("\n=== Results Summary ===")
    for i, layer in enumerate(layers):
        print(f"Layer {i}:")
        print(f"  Vertices: {layer['context'].shape[0]}")
        print(f"  Edges: {(layer['edges'][:, :, 0] > 0).sum().item()}")
        print(f"  Avg similarity: {layer['similarity'].mean():.3f}")
        
        # Show some edge details
        edge_mask = layer['edges'][:, :, 0] > 0
        if edge_mask.any():
            edge_indices = torch.nonzero(edge_mask)
            print(f"  Sample edges: {edge_indices[:3].tolist()}")
    
    print(f"\nContent store entries: {len(cortex.content_store)}")
    
    return cortex, layers

# # Run the demo
# if __name__ == "__main__":
#     cortex, layers = run_demo()

In [4]:
cortex, layers = run_demo()

Input HLLSet A:
tensor([[0., 0., 1., 0.],
        [1., 1., 0., 1.],
        [0., 1., 1., 1.],
        [0., 1., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 1., 0.]])

Input HLLSet B:
tensor([[1., 1., 1., 0.],
        [0., 1., 1., 1.],
        [0., 1., 1., 1.],
        [1., 0., 1., 0.],
        [0., 1., 0., 1.],
        [0., 0., 1., 1.]])
=== Cortex Tensor Demo ===

Layer 0: Processing 2 input HLLSets
  Context tensor shape: torch.Size([2, 8])
  Non-zero edges: 1

Layer 1: Abstracting from previous layer
  Termination: Not enough vertices for further abstraction

=== Results Summary ===
Layer 0:
  Vertices: 2
  Edges: 1
  Avg similarity: 0.833
  Sample edges: [[0, 1]]

Content store entries: 1
